In [10]:
# %%
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor
import warnings

warnings.filterwarnings('ignore')

# %%
train = pd.read_csv('train.tsv', sep='\t')
test = pd.read_csv('test.tsv', sep='\t')
train = train.loc[train.target != 0]

reviews_emmb = pd.read_csv('reviews_with_rubert-tiny2_name.csv').drop(columns=['name', 'pred'])
train = train.merge(reviews_emmb, on='id', how='left')
test = test.merge(reviews_emmb, on='id', how='left')

reviews_emmb = pd.read_csv('reviews_with_LaBSE-en-ru_comments.csv').drop(columns=['text'])
reviews_emmb.columns = ['labse_' + x if x != 'id' else x for x in reviews_emmb.columns]
train = train.merge(reviews_emmb, on='id', how='left')
test = test.merge(reviews_emmb, on='id', how='left')

embedding_cols = [col for col in train.columns if col.startswith('embedding_') or 'pred' in col]
for col in embedding_cols:
    train[col] = train[col].fillna(train[col].mean())
    test[col] = test[col].fillna(train[col].mean())

train['category'] = train['category'].astype('category')
test['category'] = test['category'].astype('category')

# %%
train['lat'] = train['coordinates'].apply(lambda x: eval(x)[1] if pd.notna(x) else np.nan)
train['lon'] = train['coordinates'].apply(lambda x: eval(x)[0] if pd.notna(x) else np.nan)

test['lat'] = test['coordinates'].apply(lambda x: eval(x)[1] if pd.notna(x) else np.nan)
test['lon'] = test['coordinates'].apply(lambda x: eval(x)[0] if pd.notna(x) else np.nan)

moscow_lat, moscow_lon = 55.7558, 37.6173
train['dist_from_center'] = np.sqrt((train['lat'] - moscow_lat)**2 + (train['lon'] - moscow_lon)**2)
test['dist_from_center'] = np.sqrt((test['lat'] - moscow_lat)**2 + (test['lon'] - moscow_lon)**2)

train['lat_lon_ratio'] = train['lat'] / (train['lon'] + 1e-5)
test['lat_lon_ratio'] = test['lat'] / (test['lon'] + 1e-5)

train['lat_lon_sum'] = train['lat'] + train['lon']
test['lat_lon_sum'] = test['lat'] + test['lon']

train['lat_lon_diff'] = train['lat'] - train['lon']
test['lat_lon_diff'] = test['lat'] - test['lon']

coord_features = ['lat', 'lon', 'dist_from_center', 'lat_lon_ratio', 'lat_lon_sum', 'lat_lon_diff']
for feat in coord_features:
    train[feat] = train[feat].fillna(train[feat].median())
    test[feat] = test[feat].fillna(test[feat].median())


In [11]:

# %%
exclude_cols = ['id', 'name', 'address', 'coordinates', 'target', 'text', 'name_clean', 'address_clean',
'name_language', 'address_street', 'address_building', 'address_korpus', 'address_city', 'labse_text', 'address_text']

feature_cols = [col for col in train.columns if col not in exclude_cols]

X = train[feature_cols]
y = train['target']
X_test = test[feature_cols]

In [ ]:

# %%
train_data_autogluon = X.copy()
train_data_autogluon['target'] = y

predictor = TabularPredictor(label='target', eval_metric='mean_absolute_error', path="AutogluonModels")
predictor.fit(train_data_autogluon, presets='medium_quality', time_limit=10*60)

ValueError: Preset 'best_qujality' was not found. Valid presets: ['best_quality', 'high_quality', 'good_quality', 'medium_quality', 'optimize_for_deployment', 'ignore_text', 'ignore_text_ngrams', 'interpretable', 'best_quality_v082', 'high_quality_v082', 'good_quality_v082', 'extreme_quality', 'tabarena', 'experimental_quality_v120']

In [5]:
predictor.leaderboard()

model  score_val          eval_metric  pred_time_val  \
0     WeightedEnsemble_L3  -0.217700  mean_absolute_error      22.332972   
1     WeightedEnsemble_L2  -0.217700  mean_absolute_error      22.332972   
2       LightGBMXT_BAG_L1  -0.219104  mean_absolute_error       1.164438   
3         LightGBM_BAG_L1  -0.219583  mean_absolute_error       0.762356   
4  RandomForestMSE_BAG_L1  -0.226680  mean_absolute_error      20.406178   

      fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  \
0  1928.756411                0.000000           0.034317            3   
1  1928.755639                0.000000           0.033545            2   
2   397.887383                1.164438         397.887383            1   
3   306.938855                0.762356         306.938855            1   
4  1223.895857               20.406178        1223.895857            1   

   can_infer  fit_order  
0       True          5  
1       True          4  
2       True          1  
3       True          2  
4       True          3

In [8]:
test_predictions = predictor.predict(X_test)

# %%
submission = pd.DataFrame({
    'id': test['id'],
    'target': test_predictions,
})

submission.to_csv('submission_autogluon.csv', index=False)
print(f"\nSubmission файл создан!")


Submission файл создан!


In [9]:
test_predictions

0       3.580522
1       3.377104
2       3.964166
3       3.325474
4       3.450991
          ...   
9271    2.897802
9272    3.821675
9273    4.045167
9274    3.510393
9275    3.753887
Name: target, Length: 9276, dtype: float32